In [1]:
import pandas as pd

In [2]:
#Carreguem l'arxiu a inspeccionar
af_desc_df = pd.read_csv("./aforaments-descriptiu.csv")

FileNotFoundError: [Errno 2] No such file or directory: './aforaments-descriptiu.csv'

In [ ]:
#Busca els valors nuls
nulls = af_desc_df.isnull().sum()
print(nulls)

Observem dues columnes per al codi del barri, amb gran nombre de valors nuls. Comprovem si la suma dels nuls de tots dos és el total de registres, ja que això indicaria que hi ha alguna errada en combinar els arxius csv que fa que la columna s'hagi duplicat i tingui el valor nul en una d'elles quan l'altre està informada.

In [ ]:
print("Total nuls columnes codi_barri: ", af_desc_df['Codi_Barri'].isnull().sum()+af_desc_df['Codi_barri'].isnull().sum())
print("Llargada dataset: ", len(af_desc_df))

Observem que ha d'haver-hi alguna columna on totes dues columnes siguin nul·les. Comprovem si coincideixen amb aquelles on Id_aforament també ho són

In [ ]:
#Seleccionem només els registres on Id_aforament no és nul. A més, aquest registre no ens interessa, donat que no podrem identificar 
#el dispositiu de mesura en no tenir número d'identificador. 
af_desc_filtered = af_desc_df[af_desc_df['Id_aforament'].notnull()]

#Tornem a comprovar les llargades dels nuls codis barri amb el total del dataset
print("Total nuls columnes codi_barri: ", af_desc_filtered['Codi_Barri'].isnull().sum()+af_desc_filtered['Codi_barri'].isnull().sum())
print("Llargada dataset: ", len(af_desc_filtered))

Continua havent més nuls, per tant hi ha algun registre on totes dues columnes son nul·les. A continuació, substituim els nuls de la columna Codi_Barri per el valor de la columna Codi_barri, i eliminem aquesta darrera. Després comprovem si queden valors nuls a la columna Codi_Barri

In [ ]:
af_desc_filtered['Codi_Barri'] = af_desc_filtered['Codi_Barri'].fillna(af_desc_filtered['Codi_barri'])
af_desc_filtered = af_desc_filtered.drop('Codi_barri', axis=1)
print("Total nuls columnes codi_barri: ", af_desc_filtered.isnull().sum())

In [ ]:
#Finalment, eliminem els dos registres amb valors nuls que queden, ja què suposen una pèrdua irrellevant de dades.
af_desc_nonull = af_desc_filtered[af_desc_filtered['Codi_Barri'].notnull()]
af_desc_nonull.to_csv("aforaments-descriptiu.csv", index=False)

El tractament dels diferents nuls dependrà del que extraiem de l'anàlisi dels valors nuls. Ara començarem amb la càrrega del següent arxiu

In [ ]:
#Carreguem l'arxiu a inspeccionar
af_det_df = pd.read_csv("./aforaments-detall.csv")
#Busca els valors nuls
nulls = af_det_df.isnull().sum()
print(nulls)

Únicament trobem dos nuls a cada columna. Comprovem si es tracta dels mateixos registres.

In [ ]:
af_det_nulls = af_det_df[af_det_df.isnull().any(axis=1)]
af_det_nulls

Efectivament, es tracta de dues fileres amb tots els valors nuls, i que per tant, directament eliminarem donat que no afectarà al nostre estudi.

In [ ]:
#Eliminem fileres amb nuls
af_det_cl = af_det_df.dropna()
#Substituim el csv amb aquest net
af_det_cl.to_csv("aforaments-detall.csv", index=False)

Carreguem ara el següent, 'cens-activitats-comercials'

In [ ]:
#Carreguem l'arxiu a inspeccionar
cac_df = pd.read_csv("./cens-activitats-comercials.csv")
#Busca els valors nuls
nulls = cac_df.isnull().sum()
print(nulls)

En primer lloc, existeix un altre dataset amb els codis de barri i noms de barri. A partir de les columnes Nom_Barri i NBARRI podem assignar gran part dels Codi_Barri. El carreguem


In [ ]:
barris_df = pd.read_csv("./unitats-administratives-barris.csv")
#Creem un diccionari amb els codis de barri i noms de barri
code_barris = dict(zip(barris_df['nom_barri'], barris_df['codi_barri']))
#Assignem aquells que sigui el Codi_Barri nul a partir de la columna CBARRI
cac_df.loc[cac_df['Codi_Barri'].isnull(), 'Codi_Barri'] = cac_df['Nom_Barri'].map(code_barris)
#Fem el mateix amb la columna NBARRI
cac_df.loc[cac_df['Codi_Barri'].isnull(), 'Codi_Barri'] = cac_df['NBARRI'].map(code_barris)
print(cac_df.isnull().sum())

Hem reduit significativament el nombre de registres sense codi de barri. Mirem ara d'assignar als nuls Codi_Barri, la columna CBARRI que pot indicar el mateix

In [ ]:
cac_df['Codi_Barri'] = cac_df['Codi_Barri'].fillna(cac_df['CBARRI'])
print(cac_df.isnull().sum())

Ara mirarem de fer el mateix amb el camp Data_Revisio, assignant-li el camp DATA

In [ ]:
cac_df['Data_Revisio'] = cac_df['Data_Revisio'].fillna(cac_df['DATA'])
print(cac_df.isnull().sum())

Tot i que encara el nombre de valors nuls és molt elevat (1/3 part del dataset no té data de revisió informada), aquests registres no ens interessen ja què no els podem assignar a cap any. Els eliminem, ja què a més hem de tenir en compte que aquest dataset el combinarem amb un altre amb informació similar d'altres anys.

In [ ]:
cac_df2 = pd.read_csv("./cens-locals-planta-baixa-act-economica.csv")
#Calculem quin seria el percentatge respecte el total dels dos datasets si es combinessin
perc_nuls = ((cac_df['Data_Revisio'].isnull().sum())/(len(cac_df)+len(cac_df2)))*100
print("El percentatge de registres sense identificar la data sobre el total és del ", perc_nuls, "%")

El percentatge continua essent força elevat, però si mirem la longitud del dataset, veiem que hi ha suficients registres per analitzar eliminant els que no tenen barri.

In [ ]:
print(len(cac_df)+len(cac_df2)-cac_df['Data_Revisio'].isnull().sum())

In [ ]:
cac_df = cac_df[cac_df['Data_Revisio'].notnull()]
cac_df = cac_df[cac_df['Codi_Barri'].notnull()]
len(cac_df)

Veiem que entre els dos datasets continuem tenint prop de 100000 registres, que són més que suficients. Pel que fa als nuls en la columna 'Codi_Grup_Activitat' no els tractarem i els deixarem tal qual. A l'hora de calcular les variables necessàries, els tractarem com si fossin activitats no típiques dels processos de gentrificació/turistificació.

In [ ]:
cac_df.to_csv("cens-activitats-comercials.csv", index=False)

Carreguem el següent dataset, 'culturailleure-bibliotequesimuseus'

In [ ]:
#Carreguem l'arxiu a inspeccionar
bim_df = pd.read_csv("./culturailleure-bibliotequesimuseus.csv")
#Busca els valors nuls
nulls = bim_df.isnull().sum()
print(nulls)

In [ ]:
#Calculem percentatge de nuls
perc_nuls = (bim_df['addresses_neighborhood_id'].isnull().sum()/len(bim_df))*100
print("El percentatge de nuls és de ", perc_nuls, "%")

Novament el nombre de nuls és mínim respecte el total de registres. Per tant, eliminem els registres nuls i guardem el csv net

In [ ]:
bim_clean = bim_df.dropna()
bim_clean.to_csv("culturailleure-bibliotequesimuseus.csv", index=False)

Els següents tres datasets són similars, donat que estan relacionats amb l'assistència a esdeveniments culturals. Carreguem els tres.

In [ ]:
#Carreguem l'arxiu a inspeccionar
dae_df = pd.read_csv("./dades-arts-esceniques.csv")
daf_df = pd.read_csv("./dades-festivals.csv")
dme_df = pd.read_csv("./dades-museus-exposicions.csv")

In [ ]:
#Busca els valors nuls en cada dataset
nulls_dae = dae_df.isnull().sum()
nulls_daf = daf_df.isnull().sum()
nulls_dme = dme_df.isnull().sum()
print("Nulls dades arts escèniques: ", nulls_dae)
print("Nulls dades festivals: ", nulls_daf)
print("Nulls dades museus: ", nulls_dme)

Hi ha dos datasets amb un nombre baix de nuls en codi barri, i que coincideix amb nombre amb nom_barri. Si fossin els mateixos registres, els eliminarem, ja que no podrem identificar-los

In [ ]:
nulls_arts = dae_df[dae_df['Codi_Barri'].isnull()]
nulls_museus = dme_df[dme_df['Codi_Barri'].isnull()]
print(nulls_arts.isnull().sum(), "total registres: ", len(nulls_arts))
print(nulls_museus.isnull().sum(), "total registres: ", len(nulls_museus))

Podem eliminar aquests registres.

In [ ]:
dae_df = dae_df[dae_df['Codi_Barri'].notnull()]
dme_df = dme_df[dme_df['Codi_Barri'].notnull()]

In [ ]:
#Mirem els valors únics de les columnes amb nulls en aquests dos datasets
print("Valors únics 'Tipus_Equipament' arts escèniques: ", dae_df['Tipus_Equipament'].unique())
print("Valors únics 'TipusEquipament' museus: ", dme_df['TipusEquipament'].unique())

Donat que aquesta columna no ens dona cap informació rellevant pel tractament de les dades, li assignarem als valors nuls el valor 'Arts escèniques' en el cas del primer dataset, i 'Museus i exposicions' en el cas del segon. Guardem aquests dos nous csv substituint els originals.

In [ ]:
dae_df['Tipus_Equipament'] = dae_df['Tipus_Equipament'].fillna("Arts escèniques")
dme_df['TipusEquipament'] = dme_df['TipusEquipament'].fillna("Museus i exposicions")
dae_df.to_csv("dades-arts-esceniques.csv", index=False)
dme_df.to_csv("dades-museus-exposicions.csv", index=False)

En el cas de dades a festivals, primer mirem els valors únics de la columna 'Espai'

In [ ]:
daf_df['Espai'].unique()

En aquest cas, la columna 'Espai' ens indica el lloc on es produeix l'event, i a partir del dataset 'equipaments-culturals.icub' podrem identificar el barri on es produeix. Per tant, en aquest cas els valors nuls no ens aporten cap informació i eliminarem els registres.

In [ ]:
daf_df = daf_df[daf_df['Espai'].notnull()]

Pel que fa a les altres columnes que presenten nuls, fan referència al nombre d'assistents de fora de Barcelona, de Barcelona i el total. En aquest cas únicament ens interessa la columna d'Assistents totals. Mirem si a partir de les altres dues es poden omplir nuls sumant assistents de fora de Barcelona i de Barcelona.

In [ ]:
daf_df['Assistents'] = daf_df['Assistents'].fillna(daf_df['Assistents_fora_de_Barcelona']+daf_df['Assistents_a_Barcelona'])
print(daf_df.isnull().sum())

Amb aquesta assignació només hem pogut assignar als valors nuls d'Assistents, aquells en que no hi havia cap nul a les altres dues columnes. Però encara podem reassignar valors, els valors no nuls de les columnes d'assistència a Barcelona, o fora de Barcelona, a Assistents quan aquesta és nul·la

In [ ]:
daf_df['Assistents'] = (daf_df['Assistents'].fillna(daf_df['Assistents_fora_de_Barcelona']))
daf_df['Assistents'] = (daf_df['Assistents'].fillna(daf_df['Assistents_a_Barcelona']))
print(daf_df.isnull().sum())

Eliminem les dues columnes sobrants, i els nuls de la columna Assistents, donat que no ens donen informació.

In [ ]:
columnes = ['Assistents_fora_de_Barcelona', 'Assistents_a_Barcelona']
daf_df = daf_df.drop(columnes, axis=1)
daf_df = daf_df.dropna()

Finalment, substituim el csv original

In [ ]:
daf_df.to_csv("dades-festivals.csv", index=False)

Ara carreguem l'arxiu d'equipaments-restaurants

In [ ]:
#Carreguem l'arxiu a inspeccionar
rest_df = pd.read_csv("./equipament-restaurants.csv")
#Imprimim els valors nuls per variable
print(rest_df.isnull().sum())

Únicament trobem un registre nul, corresponent al nom del barri, però com tenim l'identificador, no caldrà fer cap acció sobre el valor nul.

Ara carreguem l'arxiu d'habitatges d'ús turístic

In [ ]:
#Carreguem l'arxiu a inspeccionar
hut_df = pd.read_csv("./habitatges-us-turistic.csv")
#Imprimim els valors nuls per variable
print(hut_df.isnull().sum())

In [ ]:
#Mirem el percentatge que suposen, donat que sense l'identificador del barri no podem treballar, els haurem d'eliminar de totes maneres.
print("El percentatge de nuls és de :", (hut_df['CODI_BARRI'].isnull().sum()/len(hut_df))*100, "%")

Farem servir un altre cop el dataset amb els codis de barri i noms de barri. A partir de les columnes BARRI i NOM_BARRI podem assignar gran part dels CODI_BARRI. El carreguem

Assignem els codis del barri a partir del dataframe en aquells en que CODI_BARRI és nul

In [ ]:
#Creem un diccionari amb els codis de barri i noms de barri
code_barris = dict(zip(barris_df['nom_barri'], barris_df['codi_barri']))
#Assignem aquells que sigui el CODI_BARRI nul a partir de la columna BARRI
hut_df.loc[hut_df['CODI_BARRI'].isnull(), 'CODI_BARRI'] = hut_df['BARRI'].map(code_barris)
#Fem el mateix amb la columna NOM_BARRI
hut_df.loc[hut_df['CODI_BARRI'].isnull(), 'CODI_BARRI'] = hut_df['NOM_BARRI'].map(code_barris)
print(hut_df.isnull().sum())

Detectem que hi ha encara codis sense assignar. Es pot tractar perquè el nom del barri no estigui introduit de la mateixa manera que al dataset dels codis. Comparem els valors únics d'un i altre.

In [ ]:
noms_barris_oficial = list(barris_df['nom_barri'].unique())
noms_barris_hut = list(hut_df['BARRI'].unique())
noms_barris_hut2 = list(hut_df['NOM_BARRI'].unique())

Comparem les llistes de valors únics per extreure quins són els diferents. Els fem per separat les dues columnes per a fer després les conversions necessàries.

In [ ]:
barri_dif = []
nom_barri_dif = []
for i in noms_barris_hut:
    if i not in noms_barris_oficial:
        barri_dif.append(i)
for i in noms_barris_hut2:
    if i not in noms_barris_oficial:
        nom_barri_dif.append(i)
print(barri_dif)
print(nom_barri_dif)

Si comparem amb els noms oficials, tenim les següents correspondències (el segon és l'oficial):

el Poble Sec: el Poble-sec

Sant Pere Santa Caterina i la Ribera: Sant Pere, Santa Caterina i la Ribera

Vallvidrera el Tibidabo i les Planes: Vallvidrera, el Tibidabo i les Planes

Per tant, fem les substitucions que corresponguin al dataset i tornem a fer l'assignació de codis als valors nuls

In [ ]:
hut_df.loc[hut_df['BARRI']=='el Poble Sec', 'BARRI'] = 'el Poble-sec'
hut_df.loc[hut_df['BARRI']=='Sant Pere Santa Caterina i la Ribera', 'BARRI'] = 'Sant Pere, Santa Caterina i la Ribera'
hut_df.loc[hut_df['BARRI']=='Vallvidrera el Tibidabo i les Planes', 'BARRI'] = 'Vallvidrera, el Tibidabo i les Planes'
hut_df.loc[hut_df['NOM_BARRI']=='el Poble Sec', 'NOM_BARRI'] = 'el Poble-sec'

hut_df.loc[hut_df['CODI_BARRI'].isnull(), 'CODI_BARRI'] = hut_df['BARRI'].map(code_barris)
hut_df.loc[hut_df['CODI_BARRI'].isnull(), 'CODI_BARRI'] = hut_df['NOM_BARRI'].map(code_barris)

print(hut_df.isnull().sum())

Ara comprovarem que tots els registres amb CODI_BARRI nul ja no es poden deduir des d'altres camps

In [ ]:
huts_null = hut_df[hut_df['CODI_BARRI'].isnull()]
print(huts_null.isnull().sum())

Com podem comprovar, tots els registres son nuls. Per tant, els eliminem, eliminem les columnes amb el nom del barri que ja no necessitem, i guardem els canvis en el csv original

In [ ]:
cols_drop = ['BARRI', 'NOM_BARRI']
hut_df = hut_df.drop(cols_drop, axis=1)
hut_df = hut_df.dropna()
hut_df.to_csv("habitatges-us-turistic.csv", index=False)

Importem el darrer dataset on tenim valors nuls

In [ ]:
#Carreguem l'arxiu a inspeccionar
ter_df = pd.read_csv("./terrasses-comercos-vigents.csv")
#Imprimim els valors nuls per variable
print(ter_df.isnull().sum())

Novament, el nombre de nuls a CODI_BARRI i NOM_BARRI coincideix. Si es tracta dels mateixos registres, els eliminarem

In [ ]:
ter_nuls = ter_df[ter_df['CODI_BARRI'].isnull()]
print("Total registres: ", len(ter_nuls), "Nuls detectats: ", ter_nuls.isnull().sum())

Eliminem els nuls i substituim el csv original

In [ ]:
ter_df = ter_df.dropna()
ter_df.to_csv("terrasses-comercos-vigents.csv", index=False)